In [1]:
""" Training an slightly modified version of
    ResNet-18 to see effects of cutout+e_swish_2.
    
    Expected: > 92-93% accuracy
"""

from cutout_eraser import get_random_eraser
from resnet18_mod import ResNetPreAct

import keras
import keras.backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

Using TensorFlow backend.


In [2]:
# Custom activation function 2
# positive part of swish mirrored across x=1
def e_swish_2(x):
    return K.maximum(x*K.sigmoid(x), x*(2-K.sigmoid(x)))

In [3]:
act, act_name = e_swish_2, "e_swish_2"

In [4]:
model = ResNetPreAct(e_swish_2, layer1_params=(3,128,2),res_layer_params=(3,32,25),reg=5e-5)
model.summary()

C:\Users\Eric\Desktop\Programming\PYTHON\AI\TENSORFLOW\KERAS\learning_keras\ALCAIDE ACTIVATION\development\RESNET_18\resnet18_mod.py:112: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same", name="conv0", use_bias=False, strides=(2, 2), kernel_regularizer=<keras.reg..., kernel_initializer="glorot_normal")`
  name='conv0'
C:\Users\Eric\Desktop\Programming\PYTHON\AI\TENSORFLOW\KERAS\learning_keras\ALCAIDE ACTIVATION\development\RESNET_18\resnet18_mod.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (1, 1), name="conv1a", use_bias=False, kernel_initializer="glorot_normal", kernel_regularizer=<keras.reg...)`
  name=conv_name+'a'
C:\Users\Eric\Desktop\Programming\PYTHON\AI\TENSORFLOW\KERAS\learning_keras\ALCAIDE ACTIVATION\development\RESNET_18\resnet18_mod.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", name="conv1b", use_bias=False, kernel_regularizer=<keras.reg...

C:\Users\Eric\Desktop\Programming\PYTHON\AI\TENSORFLOW\KERAS\learning_keras\ALCAIDE ACTIVATION\development\RESNET_18\resnet18_mod.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", name="conv9b", use_bias=False, kernel_regularizer=<keras.reg..., kernel_initializer="glorot_normal")`
  name=conv_name+'b'
C:\Users\Eric\Desktop\Programming\PYTHON\AI\TENSORFLOW\KERAS\learning_keras\ALCAIDE ACTIVATION\development\RESNET_18\resnet18_mod.py:49: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), name="conv9c", kernel_initializer="glorot_normal", kernel_regularizer=<keras.reg...)`
  name=conv_name+'c'
C:\Users\Eric\Desktop\Programming\PYTHON\AI\TENSORFLOW\KERAS\learning_keras\ALCAIDE ACTIVATION\development\RESNET_18\resnet18_mod.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (1, 1), name="conv10a", use_bias=False, kernel_initializer="glorot_normal", kernel_regularizer=<keras.reg...)`
  

C:\Users\Eric\Desktop\Programming\PYTHON\AI\TENSORFLOW\KERAS\learning_keras\ALCAIDE ACTIVATION\development\RESNET_18\resnet18_mod.py:49: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), name="conv18c", kernel_initializer="glorot_normal", kernel_regularizer=<keras.reg...)`
  name=conv_name+'c'
C:\Users\Eric\Desktop\Programming\PYTHON\AI\TENSORFLOW\KERAS\learning_keras\ALCAIDE ACTIVATION\development\RESNET_18\resnet18_mod.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (1, 1), name="conv19a", use_bias=False, kernel_initializer="glorot_normal", kernel_regularizer=<keras.reg...)`
  name=conv_name+'a'
C:\Users\Eric\Desktop\Programming\PYTHON\AI\TENSORFLOW\KERAS\learning_keras\ALCAIDE ACTIVATION\development\RESNET_18\resnet18_mod.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", name="conv19b", use_bias=False, kernel_regularizer=<keras.reg..., kernel_initializer="glorot_normal")`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cifar (InputLayer)              (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 16, 16, 128)  3456        cifar[0][0]                      
__________________________________________________________________________________________________
bn0 (BatchNormalization)        (None, 16, 16, 128)  64          conv0[0][0]                      
__________________________________________________________________________________________________
relu0 (Activation)              (None, 16, 16, 128)  0           bn0[0][0]                        
__________________________________________________________________________________________________
conv1a (Co

In [5]:
# Data augmentation come at me
""" With 50% probability, erase 16*16 pixel squares. 
    - p=0.5 for 50% probability
    - s_l, s_h = 0.5 for only 16*16 (out of 32*32) pixels cut.
    - r_1, r_2 = 1 for squares, not rectangles
    - v_l, v_h = 0,255 since data is not normalized. Can change to 0,1 if it is.
"""
eraser = get_random_eraser(p=0.5, s_l=0.375, s_h = 0.375, r_1=1, r_2=1, v_l=0, v_h=255)

datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=10,
    fill_mode = "constant",
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True,
    vertical_flip=False,
    preprocessing_function=eraser
    )
datagen.fit(x_train)

In [ ]:
def schedule(x):
    if x<50:
        return 1e-3
    elif x<75:
        return 1e-4 
    else:
        return 1e-5

In [ ]:
#training
batch_size  = 44
epochs = 50

opt = keras.optimizers.Adam()
lr_1 = keras.callbacks.LearningRateScheduler(schedule)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print("Finished compiling")

####################
# Network training #
####################
                     
print("Gonna fit the model")
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                          epochs=epochs,verbose=1,validation_data=(x_test, y_test), callbacks=[lr_1])
print(his.history)


Finished compiling
Gonna fit the model
Epoch 1/50
1136/1136 [==============================] - 1682s 1s/step - loss: 1.8301 - acc: 0.3540 - val_loss: 1.6016 - val_acc: 0.4609
Epoch 2/50
1136/1136 [==============================] - 1669s 1s/step - loss: 1.4677 - acc: 0.5021 - val_loss: 1.4198 - val_acc: 0.5411
Epoch 3/50
1136/1136 [==============================] - 1669s 1s/step - loss: 1.2979 - acc: 0.5691 - val_loss: 1.1919 - val_acc: 0.6070
Epoch 4/50
1136/1136 [==============================] - 1669s 1s/step - loss: 1.2031 - acc: 0.6044 - val_loss: 1.2794 - val_acc: 0.6009
Epoch 5/50
1136/1136 [==============================] - 1669s 1s/step - loss: 1.1308 - acc: 0.6302 - val_loss: 1.0759 - val_acc: 0.6516
Epoch 6/50
1136/1136 [==============================] - 1669s 1s/step - loss: 1.0850 - acc: 0.6508 - val_loss: 1.1176 - val_acc: 0.6583
Epoch 7/50
1136/1136 [==============================] - 1669s 1s/step - loss: 1.0383 - acc: 0.6656 - val_loss: 0.9628 - val_acc: 0.7015
Epoch 8/5

In [ ]:
#training
batch_size  = 44
epochs = 100

opt = keras.optimizers.Adam(lr=1e-4)
lr_1 = keras.callbacks.LearningRateScheduler(schedule)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print("Finished compiling")

####################
# Network training #
####################
                     
print("Gonna fit the model")
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                          epochs=epochs,verbose=1,validation_data=(x_test, y_test), callbacks=[lr_1], initial_epoch=50)
print(his.history)

In [ ]:
print(model.evaluate(x_test, y_test))
model.save('resnet_18_eswish_cutout.h5')